In [1]:
import os
import pyspark
from pyspark.sql import SparkSession

In [2]:
rootpath = os.path.dirname(os.path.abspath(""))
datapath = os.path.join(rootpath, 'data')
print(f"datapath: {datapath}")

datapath: /home/onur/WORK/DS/repos/de_zoomcamp_nytaxi_spark/data


## Q1

In [3]:
spark = SparkSession.builder \
    .master("local[*]") \
    .appName('test') \
    .getOrCreate()

25/03/18 13:41:31 WARN Utils: Your hostname, L54Ku2004 resolves to a loopback address: 127.0.1.1; using 192.168.1.48 instead (on interface wlo1)
25/03/18 13:41:31 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/03/18 13:41:33 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [4]:
spark.version

'3.5.5'

## Q2

In [ ]:
!wget -O /home/onur/WORK/DS/repos/de_zoomcamp_nytaxi_spark/data/raw/pq/yellow_tripdata_2024-10.parquet https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2024-10.parquet

In [5]:
df_raw = spark.read \
    .parquet(os.path.join(datapath, 'raw', 'pq', 'yellow_tripdata_2024-10.parquet'))
df_raw.show()

+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+
|VendorID|tpep_pickup_datetime|tpep_dropoff_datetime|passenger_count|trip_distance|RatecodeID|store_and_fwd_flag|PULocationID|DOLocationID|payment_type|fare_amount|extra|mta_tax|tip_amount|tolls_amount|improvement_surcharge|total_amount|congestion_surcharge|Airport_fee|
+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+
|       2| 2024-10-01 00:30:44|  2024-10-01 00:48:26|              1|          3.0|         1|                 N|         162|         246|           1|       18.4|  1.0|    0.5|       1.

In [6]:
df_raw.repartition(4).write.parquet(
    os.path.join(datapath, 'spark', 'yellow_tripdata_2024-10'),
    mode='overwrite')

In [8]:
!tree -h /home/onur/WORK/DS/repos/de_zoomcamp_nytaxi_spark/data/spark/yellow_tripdata_2024-10/

[4.0K]  /home/onur/WORK/DS/repos/de_zoomcamp_nytaxi_spark/data/spark/yellow_tripdata_2024-10/
├── [ 22M]  part-00000-696f570d-cfb7-47e6-8aa9-4ac771b44a20-c000.snappy.parquet
├── [ 22M]  part-00001-696f570d-cfb7-47e6-8aa9-4ac771b44a20-c000.snappy.parquet
├── [ 22M]  part-00002-696f570d-cfb7-47e6-8aa9-4ac771b44a20-c000.snappy.parquet
├── [ 22M]  part-00003-696f570d-cfb7-47e6-8aa9-4ac771b44a20-c000.snappy.parquet
└── [   0]  _SUCCESS

0 directories, 5 files


## Q3

In [9]:
df_raw.registerTempTable('yellow_2024_10')

/home/onur/opt/spark/spark-3.5.5-bin-hadoop3/python/pyspark/sql/dataframe.py:329: FutureWarning: Deprecated in 2.0, use createOrReplaceTempView instead.
  warnings.warn("Deprecated in 2.0, use createOrReplaceTempView instead.", FutureWarning)


In [15]:
spark.sql("""
    SELECT count(*)
    FROM yellow_2024_10
    -- WHERE tpep_pickup_datetime >= '2024-10-15 00:00:00'
    -- AND tpep_dropoff_datetime < '2024-10-16 00:00:00'
    WHERE DATE(tpep_pickup_datetime) = '2024-10-15'
    """
    ).show()

+--------+
|count(1)|
+--------+
|  128893|
+--------+



## Q4

In [20]:
spark.sql("""
    SELECT
        -- tpep_pickup_datetime,
        -- tpep_dropoff_datetime,
        max(unix_timestamp(tpep_dropoff_datetime) - unix_timestamp(tpep_pickup_datetime))/3600 AS trip_duration
    FROM yellow_2024_10
    """
    ).show()

+------------------+
|     trip_duration|
+------------------+
|162.61777777777777|
+------------------+



## Q5
Answer: 4040

## Q6

In [ ]:
!wget -O /home/onur/WORK/DS/repos/de_zoomcamp_nytaxi_spark/data/raw/taxi_zone_lookup.csv https://d37ci6vzurychx.cloudfront.net/misc/taxi_zone_lookup.csv


In [21]:
fname_zones_csv = os.path.join(datapath, 'raw', 'taxi_zone_lookup.csv')
df_zones = spark.read \
    .option("header", "true") \
    .csv(fname_zones_csv)

df_zones.show()

+----------+-------------+--------------------+------------+
|LocationID|      Borough|                Zone|service_zone|
+----------+-------------+--------------------+------------+
|         1|          EWR|      Newark Airport|         EWR|
|         2|       Queens|         Jamaica Bay|   Boro Zone|
|         3|        Bronx|Allerton/Pelham G...|   Boro Zone|
|         4|    Manhattan|       Alphabet City| Yellow Zone|
|         5|Staten Island|       Arden Heights|   Boro Zone|
|         6|Staten Island|Arrochar/Fort Wad...|   Boro Zone|
|         7|       Queens|             Astoria|   Boro Zone|
|         8|       Queens|        Astoria Park|   Boro Zone|
|         9|       Queens|          Auburndale|   Boro Zone|
|        10|       Queens|        Baisley Park|   Boro Zone|
|        11|     Brooklyn|          Bath Beach|   Boro Zone|
|        12|    Manhattan|        Battery Park| Yellow Zone|
|        13|    Manhattan|   Battery Park City| Yellow Zone|
|        14|     Brookly

In [ ]:
# fname_zones_pq = os.path.join(datapath, 'spark', 'zones')
# df_zones.coalesce(1).write.parquet(fname_zones_pq, mode='overwrite')

In [32]:
df_zones.registerTempTable('zones')

In [34]:
spark.sql("""
    WITH least_popular AS (
    SELECT PULocationID,count(*) as counts
    FROM yellow_2024_10 y
    GROUP BY y.PULocationID
    ORDER BY counts ASC
    LIMIT 1
    )
    SELECT z.Zone, l.counts
    FROM least_popular l
    LEFT JOIN zones as z ON l.PULocationID = z.LocationID
    """
    ).show()


+--------------------+------+
|                Zone|counts|
+--------------------+------+
|Governor's Island...|     1|
+--------------------+------+



In [35]:
spark.sparkContext.stop()